In [1]:
import numpy as np
import cv2
import math
#Switching on the camera 
capt = cv2.VideoCapture(0)

while capt.isOpened():
#Capturing and reading frames	
    ret, frame = capt.read()

    #To get rectangular window to get data of hand gestures
    cv2.rectangle(frame, (100, 100), (300, 300), (0, 255, 0), 0)
    crop= frame[100:300, 100:300]

  #Gaussian blur
    blur = cv2.GaussianBlur(crop, (3, 3), 0)

    #BlueGreenRed to HueSaturationLightness
    hew = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

  #Creating binary image indicating skin colour to be white and the rest of the sub window black
    msk = cv2.inRange(hew, np.array([2, 0, 0]), np.array([20, 255, 255]))

   #Morphological transformation
    krnl = np.ones((5, 5))

   #Filtering out background noise
    dil = cv2.dilate(msk, krnl, iterations=1)
    ero = cv2.erode(dil, krnl, iterations=1)

  #Threshold and gaussian blur application
    fltr = cv2.GaussianBlur(ero, (3, 3), 0)
    ret, thresh = cv2.threshold(fltr, 127, 255, 0)

    #Showing thresholded image
    cv2.imshow("Threshold", thresh)

   #Contouring
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    try:
        #Maximum area contouring
        cntr = max(contours, key=lambda x: cv2.contourArea(x))

        #Bounding area creation
        x, y, w, h = cv2.boundingRect(cntr)
        cv2.rectangle(crop, (x, y), (x + w, y + h), (0, 0, 255), 0)

     #Finding convex hull	
        hull = cv2.convexHull(cntr)

        #Contour drawing 
        draw = np.zeros(crop.shape, np.uint8)
        cv2.drawContours(draw, [cntr], -1, (0, 255, 0), 0)
        cv2.drawContours(draw, [hull], -1, (0, 0, 255), 0)

        #Convexity defect application
        hull = cv2.convexHull(cntr, returnPoints=False)
        dfcts = cv2.convexityDefects(cntr, hull)

       # To find convex points of the finger tips via cosine rule to find the angle between start and end #point.
        cnt_dfcts = 0

        for i in range(dfcts.shape[0]):
            s, e, f, d = dfcts[i, 0]
            start = tuple(cntr[s][0])
            end = tuple(cntr[e][0])
            far = tuple(cntr[f][0])

            a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
            b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
            c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
            angle = (math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 180) / 3.14

            #Draw a circle if angle < 90 
            if angle <= 90:
                cnt_dfcts += 1
                cv2.circle(crop, far, 1, [0, 0, 255], -1)

            cv2.line(crop, start, end, [0, 255, 0], 2)

       #Printing the number of fingers
        if cnt_dfcts == 0:
            cv2.putText(frame, "OK", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255),2)
        elif cnt_dfcts == 1:
            cv2.putText(frame, "PEACE", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        elif cnt_dfcts == 2:
            cv2.putText(frame, "GOOD", (5, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        elif cnt_dfcts == 3:
            cv2.putText(frame, "HI", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        elif cnt_dfcts == 4:
            cv2.putText(frame, "THANK YOU", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
        else:
            pass
    except:
        pass

    #Displaying output image 
    cv2.imshow("Hand Gesture", frame)
    all_image = np.hstack((draw, crop))
    cv2.imshow('Contours', all_image)
#Closing the frames	
    if cv2.waitKey(1) == ord('q'):
        break
capt.release()
cv2.destroyAllWindows()
